In [74]:
import torch
import chess
import torch.nn as nn 
import torch.optim as optim 
import numpy as np
from torch.utils.data import DataLoader
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
from LMDBDataset import LMDBDataset

dataset = LMDBDataset("../../data/lmdb/")
loader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=0, pin_memory=True)

In [80]:
torch.set_printoptions(profile="full")

x, y = dataset[5]
print(x)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
       

In [81]:
print(y)

torch.set_printoptions(profile="default")

tensor(4505)


In [77]:
print("Ilość batchy:", len(loader))
print("ilość pozycji: ", len(loader)*loader.batch_size)

Ilość batchy: 6094
ilość pozycji:  780032


In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
import torch
from torch import nn, optim
from model import ChessPolicyNet

model = ChessPolicyNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

scaler = torch.amp.GradScaler()

epochs = 5

for epoch in range(epochs):
    
    model.train()
    running_loss = 0.0

    for batch_idx, (batch_x, batch_y) in enumerate(loader):
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast('cuda'):
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{batch_idx+1}/{len(loader)}], Loss: {running_loss/100:.4f}")
            running_loss = 0.0

    # --- Pod koniec epoki możesz zapisać checkpoint ---
    torch.save(model.state_dict(), f"chess_policy_epoch{epoch+1}.pt")


Epoch [1/5], Step [100/6094], Loss: 6.5455
Epoch [1/5], Step [200/6094], Loss: 6.0242
Epoch [1/5], Step [300/6094], Loss: 5.9279
Epoch [1/5], Step [400/6094], Loss: 5.7838
Epoch [1/5], Step [500/6094], Loss: 5.6260
Epoch [1/5], Step [600/6094], Loss: 5.4679
Epoch [1/5], Step [700/6094], Loss: 5.2823
Epoch [1/5], Step [800/6094], Loss: 5.0410
Epoch [1/5], Step [900/6094], Loss: 4.8185
Epoch [1/5], Step [1000/6094], Loss: 4.5955
Epoch [1/5], Step [1100/6094], Loss: 4.4184
Epoch [1/5], Step [1200/6094], Loss: 4.2193
Epoch [1/5], Step [1300/6094], Loss: 4.0551
Epoch [1/5], Step [1400/6094], Loss: 4.0009
Epoch [1/5], Step [1500/6094], Loss: 3.8994
Epoch [1/5], Step [1600/6094], Loss: 3.8118
Epoch [1/5], Step [1700/6094], Loss: 3.7005
Epoch [1/5], Step [1800/6094], Loss: 3.6649
Epoch [1/5], Step [1900/6094], Loss: 3.6028
Epoch [1/5], Step [2000/6094], Loss: 3.5539
Epoch [1/5], Step [2100/6094], Loss: 3.5139
Epoch [1/5], Step [2200/6094], Loss: 3.4336
Epoch [1/5], Step [2300/6094], Loss: 3.45